In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
! wget https://raw.githubusercontent.com/motazsaad/login-ds/master/datasets/IRIS.csv

--2020-07-15 05:21:12--  https://raw.githubusercontent.com/motazsaad/login-ds/master/datasets/IRIS.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4466 (4.4K) [text/plain]
Saving to: ‘IRIS.csv.1’

IRIS.csv.1          100%[===================>]   4.36K  --.-KB/s    in 0s      

2020-07-15 05:21:13 (38.6 MB/s) - ‘IRIS.csv.1’ saved [4466/4466]



In [3]:
df = pd.read_csv('IRIS.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['species'] = encoder.fit_transform((df['species']))
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [5]:
val_df = df.sample(frac=0.2, random_state=1337)
train_df = df.drop(val_df.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_df), len(val_df))
)

Using 120 samples for training and 30 for validation


In [6]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target, shuffle=True, batch_size=10):
  dataframe = dataframe.copy()
  labels = dataframe.pop(target)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [7]:
train_ds = df_to_dataset(dataframe=train_df, target='species')
val_ds = df_to_dataset(dataframe=val_df,  target='species')

In [8]:
for r in train_ds.take(1):
  print(r)

({'sepal_length': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([5.7, 6.1, 6.4, 5.5, 6.1, 7.7, 4.4, 6.7, 5.2, 6.8])>, 'sepal_width': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([4.4, 3. , 2.9, 2.4, 2.8, 3.8, 2.9, 3. , 3.4, 3.2])>, 'petal_length': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([1.5, 4.6, 4.3, 3.8, 4. , 6.7, 1.4, 5. , 1.4, 5.9])>, 'petal_width': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([0.4, 1.4, 1.3, 1.1, 1.3, 2.2, 0.2, 1.7, 0.2, 2.3])>}, <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 1, 1, 1, 2, 0, 1, 0, 2])>)


In [9]:
from tensorflow import feature_column

feature_columns = []
# numeric cols
for col in df.columns:
  if col == 'species':
    continue
  feature_columns.append(feature_column.numeric_column(col, dtype=tf.float16)) 
feature_columns

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None)]

In [10]:
model = tf.keras.models.Sequential()

In [11]:
df.species.unique()

array([0, 1, 2])

In [12]:
model.add(tf.keras.layers.DenseFeatures(feature_columns))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [13]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

12/12 [==============================] - 0s 18ms/step - loss: 0.9962 - accuracy: 0.4250 - val_loss: 0.9810 - val_accuracy: 0.1667
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 0.8749 - accuracy: 0.7583 - val_loss: 0.8110 - val_accuracy: 0.9333
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 0.7896 - accuracy: 0.7583 - val_loss: 0.6843 - val_accuracy: 0.9000
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 0.7169 - ac

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  640       
_________________________________________________________________
dense_1 (Dense)              multiple                  387       
Total params: 1,027
Trainable params: 1,027
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.evaluate(val_ds)

3/3 [==============================] - 0s 3ms/step - loss: 0.0636 - accuracy: 1.0000


[0.06360798329114914, 1.0]

In [19]:
model.predict(val_ds)

array([[9.9715281e-01, 2.8471588e-03, 2.4888644e-09],
       [9.9698132e-01, 3.0186640e-03, 5.2312932e-09],
       [9.9755609e-01, 2.4438354e-03, 2.4593334e-09],
       [9.9807906e-01, 1.9209252e-03, 1.8642432e-09],
       [5.2107644e-05, 4.4316139e-02, 9.5563173e-01],
       [2.1231035e-06, 4.3730583e-02, 9.5626730e-01],
       [1.5314244e-03, 7.6961470e-01, 2.2885391e-01],
       [9.9139321e-01, 8.6068166e-03, 2.6242821e-08],
       [1.9673580e-02, 9.7765428e-01, 2.6721235e-03],
       [5.5095359e-07, 4.7032330e-03, 9.9529618e-01],
       [1.8480875e-03, 8.5298461e-01, 1.4516731e-01],
       [2.1666136e-05, 1.9527300e-01, 8.0470532e-01],
       [9.9056810e-01, 9.4319684e-03, 2.7226736e-08],
       [1.3034103e-05, 2.4417467e-02, 9.7556949e-01],
       [9.9256754e-01, 7.4324594e-03, 2.0910381e-08],
       [4.4613462e-02, 9.5066410e-01, 4.7223964e-03],
       [2.0033399e-06, 4.2332295e-02, 9.5766574e-01],
       [9.9833703e-01, 1.6628836e-03, 9.1767965e-10],
       [7.0556125e-05, 1.284

In [20]:
print(np.argmax(model.predict(val_ds), axis=1).tolist())

[0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 0, 2, 0, 1, 2, 2, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0]


In [21]:
new_data = {
    'sepal_length': [5.0],
    'sepal_width': [1.2], 
    'petal_length': [3.5],
    'petal_width': [0.7]
}

new_data2 = {
    'sepal_length': [5.0, 5.1],
    'sepal_width': [1.2, 1.3], 
    'petal_length': [3.5, 3.4],
    'petal_width': [0.7, 0.4]
}

new_df = pd.DataFrame(data=new_data)
new_df

,sepal_length,sepal_width,petal_length,petal_width
0,5.0,1.2,3.5,0.7


In [22]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in new_df.items()}
input_dict

{'petal_length': <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[3.5]])>,
 'petal_width': <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.7]])>,
 'sepal_length': <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[5.]])>,
 'sepal_width': <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[1.2]])>}

In [23]:
model.predict(input_dict)

array([[0.00237346, 0.9220018 , 0.07562487]], dtype=float32)

In [24]:
print(np.argmax(model.predict(input_dict), axis=1).tolist())

[1]
